**Import required libraries and scripts**

In [ ]:
import os
for dir in os.listdir('/media/mario/T7/DEKOIS/'):
    print(dir)

In [3]:
import os
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.performance_calculation import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *

for dir in os.listdir('/media/mario/T7/DEKOIS/'):
    print(dir)
    software = '/home/mario/DockM8/software'
    protein_file = f'/media/mario/T7/DEKOIS/{dir}/receptor_protoss_prepared.pdb'
    ref_file = f'/media/mario/T7/DEKOIS/{dir}/crystal_ligand_protoss.sdf'
    pocket = 'reference'
    protonation = 'pkasolver'
    docking_library = f'/media/mario/T7/DEKOIS/{dir}/merged_actives_decoys.sdf'
    docking_programs = ['GNINA', 'SMINA', 'PLANTS']
    clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
    clustering_method = 'KMedoids'
    rescoring= ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'SCORCH', 'RTMScore', 'vinardo']
    id_column = 'ID'
    n_poses = 10
    exhaustiveness = 8
    parallel = 1
    ncpus = int(os.cpu_count()/2)
    #Create a temporary folder for all further calculations
    w_dir = os.path.dirname(protein_file)
    print('The working directory has been set to:', w_dir)
    create_temp_folder(w_dir+'/temp')
    
    try:
        if os.path.isfile(protein_file.replace('.pdb', '_pocket.pdb')) == False:
            if pocket == 'reference':
                pocket_definition = GetPocket(ref_file, protein_file, 8)
            elif pocket == 'dogsitescorer':
                pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')
        if os.path.isfile(w_dir+'/temp/final_library.sdf') == False:
            prepare_library(docking_library, id_column, software, protonation, ncpus)
        if os.path.isfile(w_dir+'/temp/allposes.sdf') == False:
            print('docking')
            docking(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses, ncpus)
        print('Loading all poses SDF file...')
        tic = time.perf_counter()
        all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
        toc = time.perf_counter()
        print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')
        for metric in clustering_metrics:
            if os.path.isfile(w_dir+f'/temp/clustering/{metric}_clustered.sdf') == False:
                cluster(metric, clustering_method, w_dir, protein_file, all_poses, ncpus)
        for metric in clustering_metrics:
            rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring, ncpus)
        calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)
        apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)
    except Exception as e:
        printlog(f'Failed for {dir}')
        print(e)


ace
The working directory has been set to: /media/mario/T7/DEKOIS/ace
The folder: /media/mario/T7/DEKOIS/ace/temp already exists
adrb2
The working directory has been set to: /media/mario/T7/DEKOIS/adrb2
The folder: /media/mario/T7/DEKOIS/adrb2/temp already exists
aldr
The working directory has been set to: /media/mario/T7/DEKOIS/aldr
The folder: /media/mario/T7/DEKOIS/aldr/temp already exists
andr
The working directory has been set to: /media/mario/T7/DEKOIS/andr
The folder: /media/mario/T7/DEKOIS/andr/temp already exists
cdk2
The working directory has been set to: /media/mario/T7/DEKOIS/cdk2
The folder: /media/mario/T7/DEKOIS/cdk2/temp already exists
cox1
The working directory has been set to: /media/mario/T7/DEKOIS/cox1
The folder: /media/mario/T7/DEKOIS/cox1/temp already exists
dhi1
The working directory has been set to: /media/mario/T7/DEKOIS/dhi1
The folder: /media/mario/T7/DEKOIS/dhi1/temp already exists
docking
dyr
The working directory has been set to: /media/mario/T7/DEKOIS/dy